In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# LSTM 模型

     数据结构说明：
     
     1、基本数据
        1） 
        self: Open, Close, ..., Volume , INCOME, Flag   -> 为防止某些数据过大， 统一下做log，后面其它的数据统一采用该种方式。
        lstm data : line 1: [[open,....],[open,...]] 连续N个数据，通过LSTM 连续观察特征值，再进行分类
        
     2、网结构：
        
        1）
        Seq: 
            B, N , (O,H,L,..)  
            lstm --> B, N , 256  -->  liner --> B, 128  --> B,  64 --> B, 3
    

# 准备数据

In [2]:
from stock.common.sdata import StockData
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
code = '600016'
sd = StockData(code)
data = sd.combine_income(ndays=5)
data.loc[:,['Open','Close','High','Low','Volume','Amount']] =  \
    data.loc[:,['Open','Close','High','Low','Volume','Amount']].apply(lambda x: (x-np.mean(x))/np.std(x))

data_in , target = sd.package_data(data=data)
data_in = [x.values for x in data_in]
# data_a.apply(lambda x: (x-np.min(x))/(np.max(x)-np.min(x)),axis=1)
# for item in data_in:
#     # 归一化处理 , 最大值与最小值非常容易受异常点影响，所以这种方法鲁棒性较差，只适合传统精确小数据场景。
#     item['Volume'],item['Amount'] = np.log(item['Volume']),np.log(item['Amount'])
#     item = item.apply(lambda x: (x-np.min(x))/(np.max(x)-np.min(x)),axis=0)   
    # 标准化处理， 标准化在此应该更适合, 在已有样本足够多的情况下比较稳定，适合现代嘈杂大数据场景。
                                                                       

dlen = int(len(data_in) * 0.9)
train_data, train_target = torch.from_numpy(np.array(data_in[0:dlen])).float(), torch.from_numpy(np.array(target[0:dlen])).long()
test_data, test_target = torch.from_numpy(np.array(data_in[dlen:])).float(), torch.from_numpy(np.array(target[dlen:]).squeeze()).long()


In [28]:
print(train_data)

tensor([[[-1.5702, -1.5721, -1.5901, -1.6016, -0.6972, -0.6994],
         [-1.5780, -1.5683, -1.5784, -1.5855, -0.8220, -0.7494],
         [-1.5741, -1.5454, -1.5901, -1.5855, -0.7177, -0.7062],
         [-1.5898, -1.5950, -1.6253, -1.6217, -0.7760, -0.7317],
         [-1.5819, -1.5873, -1.6018, -1.5975, -0.8134, -0.7464]],

        [[-1.5780, -1.5683, -1.5784, -1.5855, -0.8220, -0.7494],
         [-1.5741, -1.5454, -1.5901, -1.5855, -0.7177, -0.7062],
         [-1.5898, -1.5950, -1.6253, -1.6217, -0.7760, -0.7317],
         [-1.5819, -1.5873, -1.6018, -1.5975, -0.8134, -0.7464],
         [-1.5976, -1.5835, -1.5823, -1.5975, -0.8340, -0.7544]],

        [[-1.5741, -1.5454, -1.5901, -1.5855, -0.7177, -0.7062],
         [-1.5898, -1.5950, -1.6253, -1.6217, -0.7760, -0.7317],
         [-1.5819, -1.5873, -1.6018, -1.5975, -0.8134, -0.7464],
         [-1.5976, -1.5835, -1.5823, -1.5975, -0.8340, -0.7544],
         [-1.5741, -1.5492, -1.5588, -1.5734, -0.7112, -0.7036]],

        ...,

     

# 网络模型 一

In [3]:
# 模型
import torch.nn  as nn
class Net(nn.Module):
    def __init__(self, n_feature, n_hidden=256, n_target=3):
        super(Net,self).__init__()
        self.lstm = nn.LSTM(input_size=n_feature,hidden_size=n_hidden,batch_first=True)
        self.layer = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, n_target),
            nn.ReLU()
        )
    
    def forward(self, data):
        x, _ = self.lstm(data)
        x = self.layer(x)
        x = x[:,-1,:]
        return x

In [19]:
print(test_data.size())
out = net(test_data)
pred = torch.max(F.softmax(out), 1)[1]
print(pred)

torch.Size([286, 5, 6])
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])


/home/hecong/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# 训练网络
import torch.optim as optim
import torch.nn.functional as F
import os
import copy
# from tensorboardX import SummaryWriter
from stock.common.log import Logger

#   tensorboard --logdir /home/hecong/dplearn/base/log

# writer = SummaryWriter("/home/hecong/dplearn/base/log")
logger = Logger('/home/hecong/dplearn/base/log')

feature = train_data.shape[2]
dataset = TensorDataset(train_data, train_target)
train_loader = DataLoader(dataset=dataset, batch_size=64, shuffle=True)
net = Net(n_feature=feature)
if os.path.exists('net_model.pkl'):
    print('加载上次训练模型...')
    net.load_state_dict(torch.load('net_model.pkl'))
    

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=1e-3)


epcho = 10000
for epcho in range(epcho):
    run_loss = 0
    for step, (x,y) in enumerate(train_loader):
        net.train()
        optimizer.zero_grad()
        target = y.view(-1)
        out = net(x)
#         print(out)
#         pred = torch.max(F.softmax(out), 1)[1]
#         print(pred)
#         print(target)
        loss = loss_function(out,target)
        run_loss += loss
#         loss.register_hook(lambda g: print(g))
        loss.backward()
        optimizer.step()
    if epcho % 5 == 0:
        
        print('epcho {} step {} loss {:.4f}'.format(epcho,step, run_loss/step))
#         print()
        net.eval()
        validate_out = net(test_data)
        pred = torch.max(F.softmax(validate_out), 1)[1]
        correct = sum(pred.data.numpy() == test_target.data.numpy())
        print('correct number : %s (%s)' % (correct, (correct / len(test_target.data.numpy()))))
        _wts = copy.deepcopy(net.state_dict())  
        torch.save(_wts,'net_model.pkl')       
        logger.scalar_summary('loss', (run_loss/step), epcho)
        logger.scalar_summary('correct', (correct / len(test_target.data.numpy())), epcho)
        for name, param in net.named_parameters():
            logger.histo_summary(name, param.clone().cpu().data.numpy(), epcho)
            if param.grad is not None:
                logger.histo_summary(name+'/grad', param.grad.clone().cpu().data.numpy(), epcho)
        
        
        
    

epcho 0 step 40 loss 0.9936


/home/hecong/venv/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


correct number : 116 (0.40559440559440557)
epcho 5 step 40 loss 0.9750
correct number : 116 (0.40559440559440557)
epcho 10 step 40 loss 0.9642
correct number : 116 (0.40559440559440557)
epcho 15 step 40 loss 0.9470
correct number : 150 (0.5244755244755245)
epcho 20 step 40 loss 0.9379
correct number : 115 (0.4020979020979021)
epcho 25 step 40 loss 0.9385
correct number : 116 (0.40559440559440557)
epcho 30 step 40 loss 0.9399
correct number : 108 (0.3776223776223776)
epcho 35 step 40 loss 0.9424
correct number : 109 (0.3811188811188811)
epcho 40 step 40 loss 0.9376
correct number : 108 (0.3776223776223776)
epcho 45 step 40 loss 0.9518
correct number : 112 (0.3916083916083916)
epcho 50 step 40 loss 0.9326
correct number : 116 (0.40559440559440557)
epcho 55 step 40 loss 0.9286
correct number : 109 (0.3811188811188811)
epcho 60 step 40 loss 0.9250
correct number : 111 (0.3881118881118881)
epcho 65 step 40 loss 0.9306
correct number : 112 (0.3916083916083916)
epcho 70 step 40 loss 0.9318
co

In [46]:
for name, param in net.named_parameters():
    if param.grad is not None:
        print(name)
        print(param.grad.size())
    

lstm.weight_ih_l0
torch.Size([1024, 6])
lstm.weight_hh_l0
torch.Size([1024, 256])
lstm.bias_ih_l0
torch.Size([1024])
lstm.bias_hh_l0
torch.Size([1024])
layer.0.weight
torch.Size([512, 256])
layer.0.bias
torch.Size([512])
layer.2.weight
torch.Size([256, 512])
layer.2.bias
torch.Size([256])
layer.4.weight
torch.Size([64, 256])
layer.4.bias
torch.Size([64])
layer.6.weight
torch.Size([3, 64])
layer.6.bias
torch.Size([3])


In [45]:
writer.close()

In [10]:
# dd = iter(train_loader)
# x, y = next(dd)
# out = net(x)
# print(out.size())

print(train_data.size())


torch.Size([2566, 5, 6])
